In [1]:
%matplotlib inline
import shutil
from  ast import literal_eval
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

from common import constant
from common import rle

In [2]:
SAVE_DST = Path(constant.DATASET_PATH) / 'yolov8_txt_data'
SAVE_DST.mkdir(parents=True, exist_ok=True)

TRAIN_DST = Path(constant.DATASET_PATH) / 'yolov8'
TRAIN_DST.mkdir(parents=True, exist_ok=True)

In [3]:
df = pd.read_csv(constant.TRAIN_CSV_PATH)

In [3]:
# Verticesは1つであるはずだが、そうでないもののインデックスを取得
irregulars = []
for i, row in df.iterrows():
    if len(literal_eval(row['Vertices'])) != 1:
        irregulars.append(i)
        print(i)

16122
16142
16143
16144
16145
16146
16147
16148
16149
23532
24348
24349
24350
24351
24352
24353
24354
24355
24356


In [5]:
# 以下のインスタンスはアノテーションみすっぽい(下記の可視化で目視確認)
miss_annt_instance_indexes = [16142, 16143, 16144, 16145, 16146, 16147, 16148, 16149]

In [ ]:
# for i in irregulars:
#     print(i)
#     row = df.iloc[i]
#     encoded = rle.rle_decode(row['rle'], shape=(row['height'], row['width']))
#     plt.imshow(encoded)
#     plt.show()
#     vertices = literal_eval(row['Vertices'])
#     for vertice in vertices:
#         img = plt.imread(constant.TRAIN_IMG_ROOT / row['image'])
#         cv2.fillPoly(img, [np.array(vertice)], (255, 0, 0))
#         plt.imshow(img)
#         plt.show()

In [40]:
img_list = df['image'].unique().tolist()
for target_img in img_list:
    target_df = df[df['image'] == target_img]
    result = []
    for i, row in target_df.iterrows():
        if i in miss_annt_instance_indexes:
            continue
        divide_arr = np.array([row['width'], row['height']])
        vertices = (np.array(literal_eval(row['Vertices'])[0]) / divide_arr).reshape(-1).tolist()
        result.append([0] + vertices)
    txt_name = target_img.replace('.png', '.txt')
    with open(str(SAVE_DST / txt_name), 'w') as f:
        for i in range(len(result)):
            line = *(result[i]),
            f.write(('%g ' * len(line)).rstrip() % line + '\n')

In [4]:
img_dst_dir = TRAIN_DST / 'images' / 'train'
img_dst_dir.mkdir(parents=True, exist_ok=True)
label_dst_dir = TRAIN_DST / 'labels' / 'train'
label_dst_dir.mkdir(parents=True, exist_ok=True)

img_list = df['image'].unique().tolist()
for target in tqdm(img_list):
    img_src = constant.TRAIN_IMG_ROOT / target
    img_dst = img_dst_dir / target
    shutil.copy(img_src, img_dst)
    
    txt_name = target.replace('.png', '.txt')
    txt_src = SAVE_DST / txt_name
    txt_dst = label_dst_dir / txt_name
    shutil.copy(txt_src, txt_dst)

100%|█████████████████████████████████████████| 122/122 [00:07<00:00, 17.02it/s]


In [12]:
img_dst

PosixPath('/work/dataset/yolov8/images/train/pcb_32b_cc_12.png/pcb_9f_cc_5.png')